In [2]:
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud, STOPWORDS

In [3]:
pip install -U spacy

     |████████████████████████████████| 6.2 MB 4.5 MB/s 
     |████████████████████████████████| 181 kB 22.2 MB/s 
     |████████████████████████████████| 10.1 MB 1.3 MB/s 
     |████████████████████████████████| 457 kB 68.1 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 653 kB 60.8 MB/s 
     |████████████████████████████████| 58 kB 7.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Un

In [4]:
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 460.3 MB 26 kB/s 
     |████████████████████████████████| 51 kB 185 kB/s 
     |████████████████████████████████| 1.1 MB 10.0 MB/s 
     |████████████████████████████████| 3.8 MB 65.9 MB/s 
     |████████████████████████████████| 84 kB 4.4 MB/s 
     |████████████████████████████████| 880 kB 59.0 MB/s 
     |████████████████████████████████| 6.6 MB 56.1 MB/s 
     |████████████████████████████████| 596 kB 66.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3a0c288a48e26adb17a0175809cd3f0b6c1e907828dc542e637e8d5c2b94ab88
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
✔ Download and installation successful
You can now load the package via spacy.load('en_core_we

In [ ]:
#gpu check

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
import spacy

In [7]:
nlp_best = spacy.load("/content/drive/MyDrive/Capp_Model/model-best")

/usr/local/lib/python3.7/dist-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
!unzip /content/drive/MyDrive/Capp_diseases/diseases_selected.zip

Archive:  /content/drive/MyDrive/Capp_diseases/diseases_selected.zip
   creating: diseases_selected/acne/
  inflating: diseases_selected/acne/emedicine_acne.txt  
  inflating: diseases_selected/acne/healthline_acne.txt  
  inflating: diseases_selected/acne/tata1mg_acne.txt  
   creating: diseases_selected/arthritis/
  inflating: diseases_selected/arthritis/Arthritis.txt  
  inflating: diseases_selected/arthritis/emedicine_arthritis.txt  
  inflating: diseases_selected/arthritis/tata1mg_arthritis-117.txt  
   creating: diseases_selected/asthama/
  inflating: diseases_selected/asthama/Asthma.txt  
  inflating: diseases_selected/asthama/emedicine_asthama.txt  
  inflating: diseases_selected/asthama/everyday_asthma_guide.txt  
  inflating: diseases_selected/asthama/tata1mg_asthma-116.txt  
   creating: diseases_selected/breast cancer/
  inflating: diseases_selected/breast cancer/Breast Cancer_healthline (1).txt  
  inflating: diseases_selected/breast cancer/Breast Cancer_hopkinsmedicine (1

In [9]:
global_path = "/content/diseases_selected/"
contents = []
for folder in ['acne', 'arthritis', 'asthama', 'breast cancer', 'bronchitis','chickenpox','conjunctivitis','dengue','diabetes',
               'diarrhea','migrane','pneumonia','scabies','spondylosis','malaria','hyperthyroidism','hypothyroidism','thrombocytopenia','tuberculosis']:
  folder_path = global_path + folder + "/"
  files = os.listdir(folder_path)
  files = sorted(files)
  for file in files:
    file_path = folder_path + file
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
      try:
        description = f.read()
        contents.append((folder,description))
      except:
        print(file_path)
        continue

In [10]:
contents[44]

('hyperthyroidism',
 'What Is It?\nSymptoms\nCauses\nDiagnosis\nTreatment\nGuide\nWhat Is Hyperthyroidism?\nSome people with hyperthyroidism may have no symptoms. When symptoms of hyperthyroidism occur, they may include anxiety, irritability, trouble sleeping, and others.\nHyperthyroidism (also called overactive thyroid) occurs when the thyroid gland in the neck produces too much thyroid hormone and increases the body\x92s metabolism. \nHyperthyroidism is not the same as hypothyroidism, which is when a person produces too little thyroid hormone. \nWhat Are Symptoms of Hyperthyroidism?\nSome people with hyperthyroidism may have no symptoms. When symptoms of hyperthyroidism occur, they may include:\nAnxiety\nIrritability\nTrouble sleeping\nWeakness (especially in the upper arms and thighs, which can make it difficult to lift heavy things or climb stairs)\nTremors of the hands\nSweating more than usual\nHeat intolerance\nFast or uneven heartbeats\nTiredness/fatigue \nWeight loss even with

In [11]:
contents[0]

('acne',
 'What Is It?\nSymptoms and Causes\nWhen to See the Doctor\nDiagnosis\nHome Remedies\nTreatment\nPrognosis\nGuide\nFacts on Acne\nShare Your Story\nAcne is not caused by eating "fast" foods, chocolate, or high-fat foods.\nAcne is an inflammatory rash that is almost universal among individuals going through puberty in industrialized societies. It can, however, occur at all ages. Typical acne affects the skin of the face, chest, and back and rarely the neck and upper arms of teenagers and young adults.\nBecause acne typically occurs during a time of dramatic physical and psychological changes associated with the development of one\'s body image, it can exacerbate social withdrawal and even depression. Left untreated, severe acne can lead to disfiguring scarring which can be difficult to treat.\nSeveral myths exist about acne. Here are the facts.\nAcne is not a result of poor hygiene or infrequent washing. In other words, acne does not result from too much dirt on the skin or in 

In [12]:
df = pd.DataFrame.from_records(contents, columns = ['disease', 'description'])
df

,disease,description
0,acne,What Is It?\nSymptoms and Causes\nWhen to See ...
1,acne,"Acne: Treatment, Types, Causes, Prevention, an..."
2,acne,"Overview\nAcne is a common condition, which mo..."
3,arthritis,ï»¿Arthritis | Johns Hopkins Medicine Skip Nav...
4,arthritis,ï»¿Facts\nSymptoms & Signs\nCauses\nRisk Facto...
5,arthritis,ï»¿Overview\nArthritis is a common condition t...
6,asthama,Asthma | Johns Hopkins Medicine Skip Navigatio...
7,asthama,What Is\nTypes\nCommon Causes\nSymptoms\nTreat...
8,asthama,Medically Reviewed\nAsthma is a common chronic...
9,asthama,Overview\nAsthma is a long term condition in w...


In [13]:
df.disease.value_counts()

asthama             4
acne                3
diabetes            3
spondylosis         3
scabies             3
pneumonia           3
arthritis           3
diarrhea            3
dengue              3
conjunctivitis      3
chickenpox          3
bronchitis          3
breast cancer       3
tuberculosis        3
migrane             2
malaria             2
hyperthyroidism     2
hypothyroidism      1
thrombocytopenia    1
Name: disease, dtype: int64

In [14]:
df.disease.nunique()

19

In [15]:
df.head(20)

,disease,description
0,acne,What Is It?\nSymptoms and Causes\nWhen to See ...
1,acne,"Acne: Treatment, Types, Causes, Prevention, an..."
2,acne,"Overview\nAcne is a common condition, which mo..."
3,arthritis,ï»¿Arthritis | Johns Hopkins Medicine Skip Nav...
4,arthritis,ï»¿Facts\nSymptoms & Signs\nCauses\nRisk Facto...
5,arthritis,ï»¿Overview\nArthritis is a common condition t...
6,asthama,Asthma | Johns Hopkins Medicine Skip Navigatio...
7,asthama,What Is\nTypes\nCommon Causes\nSymptoms\nTreat...
8,asthama,Medically Reviewed\nAsthma is a common chronic...
9,asthama,Overview\nAsthma is a long term condition in w...


In [16]:
data = df.copy()

In [17]:
only_description = data.description

In [18]:
#Basic preprocessing
corpus = [re.sub("\\n\\n", " ", s) for s in only_description]
corpus = [re.sub("\n", " ", s) for s in corpus]
corpus = [x.lower() for x in corpus]
corpus = [re.sub("[^a-z' ]+", "", x) for x in corpus]
corpus = [x.strip() for x in corpus]

In [19]:
corpus[44]

"what is it symptoms causes diagnosis treatment guide what is hyperthyroidism some people with hyperthyroidism may have no symptoms when symptoms of hyperthyroidism occur they may include anxiety irritability trouble sleeping and others hyperthyroidism also called overactive thyroid occurs when the thyroid gland in the neck produces too much thyroid hormone and increases the bodys metabolism  hyperthyroidism is not the same as hypothyroidism which is when a person produces too little thyroid hormone  what are symptoms of hyperthyroidism some people with hyperthyroidism may have no symptoms when symptoms of hyperthyroidism occur they may include anxiety irritability trouble sleeping weakness especially in the upper arms and thighs which can make it difficult to lift heavy things or climb stairs tremors of the hands sweating more than usual heat intolerance fast or uneven heartbeats tirednessfatigue  weight loss even with a normal or increased appetite  frequent bowel movements  swelling

In [ ]:
#tag = []
#for doc in nlp_best.pipe(corpus, disable=["tagger", "parser"]):
   # tag.append([ent.text for ent in doc.ents])


In [ ]:
#import pickle
#with open('tag_pickle','wb') as f:
 # pickle.dump(tag,f)
 

In [ ]:
#with open('tag_pickle','wb') as f:
 # pickle.dump(tag,f)
 #with open('tag_pickle','rb') as f:
 # tag = pickle.load(f)

In [20]:
import pickle

In [21]:
with open('/content/drive/MyDrive/Capp_Model/tag_pickle','rb') as f:
 tag = pickle.load(f)

In [22]:
tag

[['acne',
  'acne',
  'acne',
  'acne',
  'skin',
  'neck',
  'young adults',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'skin',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'skin',
  'skin',
  'swelling',
  'redness',
  'skin',
  'acne',
  'redness',
  'skin',
  'acne',
  'women',
  'skin',
  'acne',
  'acne',
  'acne',
  'female',
  'acne',
  'acne',
  'skin',
  'palms',
  'skin',
  'skin',
  'skin',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'skin',
  'skin',
  'acne',
  'acne',
  'skin',
  'acne',
  'skin',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'fever',
  'swelling',
  'skin',
  'skin',
  'skin',
  'acne',
  'acne',
  'acne',
  'skin',
  'skin',
  'acne',
  'acne',
  'acne',
  'skin',
  'acne',
  'acne',
  'acne',
  'acne',
  'skin',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'acne',
  'skin',
  'acne',
  'acne',
  'skin',
  'acne',
  'acne',
  'acne',
  'skin',
  'acne',
  'acne',
  '

In [23]:
print(tag)

[['acne', 'acne', 'acne', 'acne', 'skin', 'neck', 'young adults', 'acne', 'acne', 'acne', 'acne', 'acne', 'skin', 'acne', 'acne', 'acne', 'acne', 'acne', 'skin', 'skin', 'swelling', 'redness', 'skin', 'acne', 'redness', 'skin', 'acne', 'women', 'skin', 'acne', 'acne', 'acne', 'female', 'acne', 'acne', 'skin', 'palms', 'skin', 'skin', 'skin', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'skin', 'skin', 'acne', 'acne', 'skin', 'acne', 'skin', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'fever', 'swelling', 'skin', 'skin', 'skin', 'acne', 'acne', 'acne', 'skin', 'skin', 'acne', 'acne', 'acne', 'skin', 'acne', 'acne', 'acne', 'acne', 'skin', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'skin', 'acne', 'acne', 'skin', 'acne', 'acne', 'acne', 'skin', 'acne', 'acne', 'acne', 'skin', 'acne', 'acne', 'skin', 'skin', 'acne', 'acne', 'skin', 'acne', 'isotretinoin', 'depression', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'acne', 'face', 'ches

In [24]:
len(tag)

51

In [25]:
res = []
for j in range(0,len(tag)):
  for i in tag[j]:
    if i not in res:
      res.append(i)

In [26]:
k = 0
dict = {}
for i in res:
  dict[i] = k
  k = k+1

In [ ]:
dict

In [27]:
tfidf = TfidfVectorizer(stop_words = STOPWORDS, ngram_range= (1,2), max_df = 0.7, min_df = 30, max_features = 200, token_pattern="[a-z']+", vocabulary = dict)

In [28]:
tfidf_mat = tfidf.fit_transform(corpus)

In [29]:
print(len(tfidf.vocabulary_))

1022


In [30]:
#tfidf.vocabulary_

In [31]:
tfidf_mat


<51x1022 sparse matrix of type '<class 'numpy.float64'>'
	with 3324 stored elements in Compressed Sparse Row format>

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import cohen_kappa_score, make_scorer

In [33]:
def kappa_score(y_act, y_pred):
  return cohen_kappa_score(y_act, y_pred)

kappa = make_scorer(kappa_score)

In [34]:
lr = LogisticRegression(max_iter = 1000)

In [35]:
cross_val_score(lr, X= tfidf_mat, y = data.disease, cv = 4, scoring = kappa)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  UserWarning,


array([1.        , 0.91666667, 0.83544304, 0.90977444])

In [36]:
lr_model = lr.fit(tfidf_mat, data.disease)

In [37]:
y_pred_train_lr = lr_model.predict(tfidf_mat)

In [38]:
y_pred_train_lr

array(['acne', 'acne', 'acne', 'arthritis', 'arthritis', 'arthritis',
       'asthama', 'asthama', 'asthama', 'asthama', 'breast cancer',
       'breast cancer', 'breast cancer', 'bronchitis', 'bronchitis',
       'bronchitis', 'chickenpox', 'chickenpox', 'chickenpox',
       'conjunctivitis', 'conjunctivitis', 'conjunctivitis', 'dengue',
       'dengue', 'dengue', 'diabetes', 'diabetes', 'diabetes', 'diarrhea',
       'diarrhea', 'diarrhea', 'migrane', 'migrane', 'pneumonia',
       'pneumonia', 'pneumonia', 'scabies', 'scabies', 'scabies',
       'spondylosis', 'spondylosis', 'spondylosis', 'malaria', 'malaria',
       'hyperthyroidism', 'hyperthyroidism', 'hyperthyroidism', 'dengue',
       'tuberculosis', 'tuberculosis', 'tuberculosis'], dtype=object)

In [39]:
from sklearn.metrics import plot_confusion_matrix, roc_curve, accuracy_score, f1_score

In [40]:
accuracy_score(data.disease, y_pred_train_lr)

0.9607843137254902

In [41]:
def text(x):
  l1 = []
  text1 = x
  l1.append(text1)
  corpus_test = [re.sub("\\n\\n", " ", s) for s in l1]
  corpus_test = [re.sub("\n", " ", s) for s in corpus_test]
  corpus_test = [x.lower() for x in corpus_test]
  corpus_test = [re.sub("[^a-z' ]+", "", x) for x in corpus_test]
  corpus_test = [x.strip() for x in corpus_test]
  return(corpus_test)

In [42]:
def predict_disease(d):
  n = text(d)
  tfidf_mat_test = tfidf.fit_transform(n)
  y_pred_test = lr_model.predict(tfidf_mat_test)
  y_pred_test_proba = lr_model.predict_proba(tfidf_mat_test)
  dis =  y_pred_test
  print(np.max(y_pred_test_proba))
  disease_check(dis,d)
  #print(" \n There is {prob}% chance that you are suffering from {dis}. Kindly consult with a doctor for further diagnosis. Thank You".format(prob = round(np.max(y_pred_test_proba)*100,2), dis = y_pred_test ))
  #return y_pred_test, y_pred_test_proba

In [43]:
def symptoms():
  d0 = input("Welcome ! Kindly mention your name, age and gender : ")
  d1 = input("\n Hello !, What Symptoms are you having : ")
  d2 = input("\n Please explain in more details : ")
  d = d0 + ' '+ d1 +' '+ d2
  predict_disease(d)

In [44]:
def disease_check(dis,d):
  if(dis == "acne"):
    print("Are you suffering from any of these symptoms like blackheads, whiteheads, redness in skin, swelling. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == "arthritis"):
    print("Are you suffering from any of these symptoms like swelling of the joint, pain while walking, weightloss, redness in eyes. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'asthma'):
    print("Are you suffering from any of these symptoms like cough, runny nose, shortness of breath, fever, dry coughing. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'breast cancer'):
    print("Are you suffering from any of these symptoms like breast lumps, red or discolored pitted skin, swelling in all or part of your breast, thickening that feels different from surrounding tissue. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'bronchitis'):
    print("Are you suffering from any of these symptoms like breathlessness, chest or shoulder pain, chest tightness, high blood pressure, shortness of breath. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'chickenpox'):
    print("Are you suffering from any of these symptoms like headache, high fever, loss of appetite, rash on the eyes. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'conjunctivitis'):
    print("Are you suffering from any of these symptoms like burning sensation in your eyes, burning and itching in the eyes, fever, pink or red discoloration to the whites of the eyes. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'dengue'):
    print("Are you suffering from any of these symptoms like high fever, headache, joint pain, vomiting, shivering . If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'diabetes'):
    print("Are you suffering from any of these symptoms like shakiness, shortness of breath, sudden losss of weight, urinary tract infection. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'diarrhea'):
    print("Are you suffering from any of these symptoms like muscle cramps, loose and watery stools, abdominal or stomach pain, bloody stools. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'migrane'):
    print("Are you suffering from any of these symptoms like intense headache, head trauma, head pain, dizziness. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'pneumonia'):
    print("Are you suffering from any of these symptoms like decreased sense of smell, decreased sense of taste, chest pain, shortness of breath. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'scabies'):
    print("Are you suffering from any of these symptoms like intense itching, severe itching sensation in the affected areas of the skin. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'spondylosis'):
    print("Are you suffering from any of these symptoms like back or neck pain, buttock pain, headache, low back pain, weakness in the arms. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'malaria'):
    print("Are you suffering from any of these symptoms like abdominal pain, cough, high body temperature, muscle aches, sudden feeling of cold and shivering. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'hyperthyroidism'):
    print("Are you suffering from any of these symptoms like anxiety, bulging eyes, erectile dysfunction, excess sweating, intolerance towards heat, sleep disorders, swelling of the neck. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here")
  elif (dis == 'hypothyroidism'):
    print("Are you suffering from any of these symptoms like blurred vision, decreased appetite, dry rough skin, depression, lethargy, muscle aching, weakness. Also you may also mention symptoms not mentioned here")
  elif (dis == 'thrombocytopenia'):
    print("Are you suffering from any of these symptoms like bleeding from gums, bleeding from the anus, bleeding of the nose, fatigue blood in urine. Also you may also mention symptoms not mentioned here")
  elif (dis == 'tuberculosis'):
    print("Are you suffering from any of these symptoms like difficulty breathing, dry cough, chest pain, coughing, vomiting, fatigue darkcolored urine. Also you may also mention symptoms not mentioned here")
 

  d5 = input()
  d6 = d +' '+ d5
  m = text(d6)
  tfidf_mat_test_1 = tfidf.fit_transform(m)
  y_pred_test_1 = lr_model.predict(tfidf_mat_test_1)
  print("You are suffering from", y_pred_test_1)

In [46]:
symptoms()

Welcome ! Kindly mention your name, age and gender : Avirup 25 male

 Hello !, What Symptoms are you having : I have blood in my stools

 Please explain in more details : I have stomach pain 
0.07326013916004412
Are you suffering from any of these symptoms like back or neck pain, buttock pain, headache, low back pain, weakness in the arms. If Yes, Please mention the symptoms. Also you may also mention symptoms not mentioned here
neck pain and back pain
You are suffering from ['spondylosis']


In [ ]:
symptoms()

In [ ]:
symptoms()

In [ ]:
predict_disease("I am having small lumps in breast in and fever ")

In [ ]:
#predict_disease("I am having pain in my left knee joint from last few days. Also I am having pain while walking")

In [ ]:
#predict_disease("I am coughing from last few days, wheezing and difficulty in breathing in night. I am also feeling chest tightness")

In [ ]:
#I have feeling thirsty and hungry. I am also having frequent urination. My vision gets blurry sometimes. I have glucose in blood

In [ ]:
#import pickle